# CDC COVID-19 Case Surveillance Public Use Data 2020 Analysis

#### Megan Hoang | HUT Script | 4-7-2022

> Data from https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf

***

In [ ]:
# import all necessary modules
import pandas as pd 
import numpy as np 
import sqlite3 # for SQL queries
import csv 
import matplotlib 
from matplotlib import pyplot as plt # import matplotlib.pyplot as plt
from matplotlib import cm #Colormap
import seaborn as sns # visualization
import os # directory
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression


In [ ]:
# set our directory to the SSD
print(os.getcwd())
path = "/Volumes/Extreme SSD/Megan Windows Backup 1.6.2022/Honors Undergraduate Thesis/Analysis/Data/CDC/Case Surveillance Public Use Data/"
os.chdir(path)
print(os.getcwd())

# # read in the CSV to see if we can access it properly -- the dataset is too large, so processing using pandas "chunks"
# data = pd.read_csv("COVID-19_Case_Surveillance_Public_Use_Data.csv", low_memory=False)
# # print(data.head())

# for chunk in pd.read_csv("COVID-19_Case_Surveillance_Public_Use_Data.csv", chunksize=10):
#     print(chunk)

In [ ]:
# now that we can access the data, let's set up the database:

# set our directory
print(os.getcwd())
path = "/Users/meganhoang/Desktop/"
os.chdir(path)
print(os.getcwd())


con = sqlite3.connect('cdc.db')
cur = con.cursor()

In [ ]:
print(os.getcwd())
path = "/Volumes/Extreme SSD/Megan Windows Backup 1.6.2022/Honors Undergraduate Thesis/Analysis/Data/CDC/Case Surveillance Public Use Data/"
os.chdir(path)
print(os.getcwd())

cur.execute("""create table CDC
            (cdc_case_earliest_dt   DATETIME,
            cdc_report_dt           DATETIME,
            pos_spec_dt             DATETIME,
            onset_dt                DATETIME,
            status                  TEXT,
            sex                     TEXT,
            age                     TEXT,
            race                    TEXT,
            hosp                    TEXT,
            icu                     TEXT,
            death                   TEXT,
            medcond                 TEXT)""")

# read the csv into the database
file = open('COVID-19_Case_Surveillance_Public_Use_Data.csv') 
data = csv.reader(file)
cur.executemany('insert into CDC values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', data)
print("success!!")

In [ ]:
# let's store the variables I want to query in a string:
# dummy variables: sex: region (West omitted) 1 = female
# Age naming bracket is as follows: Child (0-9), Youth (10-19), Adult (20-59), Senior (60+)


select = """
        select
            cdc_report_dt,
            case
                when status = 'Laboratory-confirmed case' then 1
                else 0
            end as confirmed_case,
            
            case
                when sex = 'Female' then 1
                when sex = 'Male' then 0
                else 99999
            end as female,
            case
                when sex = 'Female' then 0
                when sex = 'Male' then 1
                else 99999
            end as male,

            case
                when age = '0 - 9 Years' then 1
                else 0
            end as child,
            case
                when age = '10 - 19 Years' then 1
                else 0
            end as youth,
            case
                when age = '20 - 39 Years' then 1
                when age = '40 - 49 Years' then 1
                when age = '50 - 59 Years' then 1
                else 0
            end as adult,
            case
                when age = '60 - 69 Years' then 1
                when age = '70 - 79 Years' then 1
                when age = '80 + Years' then 1
                else 0
            end as senior,

            case
                when race = 'White, Non-Hispanic' then 1
                else 0
            end as white,
            case
                when race = 'Black, Non-Hispanic' then 1
                else 0
            end as black,
            case
                when race = 'Hispanic/Latino' then 1
                else 0
            end as hisp,
            case
                when race = 'American Indian/Alaska Native, Non-Hispanic' then 1
                else 0
            end as native,
            case
                when race = 'Asian, Non-Hispanic' then 1
                when race = 'Native Hawaiian/Other Pacific Islander, Non-Hispanic' then 1
                else 0
            end as asian,

            case
                when hosp = 'Yes' then 1
                when hosp = 'No' then 0
            end as hosp,
            
            case
                when icu = 'Yes' then 1
                else 0
            end as icu,

            case
                when death = 'Yes' then 1
                when death = 'No' then 0
            end as death,
            
            case
                when medcond = 'Yes' then 1
                when medcond = 'No' then 0
            end as medcond
        """

# edit the SQL query to clean the data and omit "unknown" values per CDC codebook
# remove = """ and sex != 'Unknown' and sex != 'Other' and sex != 'Missing' and sex != 'NA' """

remove = """ and sex != 'Unknown' and sex != 'Other' and sex != 'Missing' and sex != 'NA'
        and age != 'Missing' and age != 'NA'
        and race != 'Unknown' and race != 'Missing' and race != 'NA'
        and hosp != 'Unknown' and hosp != 'Missing'
        and icu != 'Unknown' and icu != 'Missing'
        and death != 'Missing' and death != 'Unknown'
        and medcond != 'Unknown' and medcond != 'Missing'"""

# remove = ""

In [ ]:
# cur.execute("select * from CDC where race_ethnicity_combined like 'Asian, Non-Hispanic'")
# for row in cur.fetchall():
#  print(row)

# Use the commit method to save changes. 
con.commit()

In [ ]:
df_query = pd.read_sql_query(select + "from CDC where status = 'Laboratory-confirmed case'" + remove, con)
df_query.describe()

In [ ]:
df_query.dropna()
df_query.head()
# df_query.describe(include='all')

#### Visualizations
* Number of Cases by Race
* Hospitalizations by Race
* ICU Admittance by Race
* Deaths by Race

In [ ]:
# Number of Cases by Race

val_counts = []

for col in ['white', 'black', 'hisp', 'native', 'asian']:
    count = df_query[col].value_counts() 
    val_counts.append(count[1] / (count[1] + count[0]) * 1000)

import matplotlib.pyplot as plt; plt.rc("font", size=12)
y_pos = np.arange(len(['white', 'black', 'hisp', 'native', 'asian']))

p = reversed(sns.color_palette('Blues_d', n_colors=5))
sns.barplot(y_pos, val_counts, palette = p)

plt.xticks(y_pos, ['white', 'black', 'hisp', 'native', 'asian'])
plt.ylabel('Number of Cases')
plt.xlabel('Race')
plt.title('Number of Cases by Race')

plt.show()

In [ ]:
# Proportion of Hospitalizations by Race

val_counts = []

for col in ['white', 'black', 'hisp', 'native', 'asian']:
    counts_df = df_query.groupby(col)['hosp'].value_counts()
    try:
        print(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
        val_counts.append(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
    except:
        val_counts.append(0)

import matplotlib.pyplot as plt; plt.rc("font", size=12)
y_pos = np.arange(len(['white', 'black', 'hisp', 'native', 'asian']))

p = reversed(sns.color_palette('Blues_d', n_colors=5))
sns.barplot(y_pos, val_counts, palette = p)
# plt.bar(y_pos, val_counts, align='center', alpha=0.5)
plt.xticks(y_pos, ['white', 'black', 'hisp', 'native', 'asian'])
plt.ylabel('Proportion of Hospitalizations')
plt.xlabel('Race')
plt.title('Proportion of Hospitalizations by Race')

plt.show()

In [ ]:
# Proportion of ICU Cases by Race

val_counts = []

for col in ['white', 'black', 'hisp', 'native', 'asian']:
    counts_df = df_query.groupby(col)['icu'].value_counts()
    try:
        print(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
        val_counts.append(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
    except:
        val_counts.append(0)

import matplotlib.pyplot as plt; plt.rc("font", size=12)
y_pos = np.arange(len(['white', 'black', 'hisp', 'native', 'asian']))

p = reversed(sns.color_palette('Blues_d', n_colors=5))
sns.barplot(y_pos, val_counts, palette = p)
# plt.bar(y_pos, val_counts, align='center', alpha=0.5)
plt.xticks(y_pos, ['white', 'black', 'hisp', 'native', 'asian'])
plt.ylabel('Proportion of ICU Admissions')
plt.xlabel('Race')
plt.title('Proportion of ICU Admissions by Race')

plt.show()

In [ ]:
# Proportion of Deaths by Race

val_counts = []

for col in ['white', 'black', 'hisp', 'native', 'asian']:
    counts_df = df_query.groupby(col)['death'].value_counts()
    try:
        print(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
        val_counts.append(counts_df[1][1] / (counts_df[1][1] + counts_df[1][0]) * 1000)
    except:
        val_counts.append(0)

import matplotlib.pyplot as plt; plt.rc("font", size=12)
y_pos = np.arange(len(['white', 'black', 'hisp', 'native', 'asian']))

p = reversed(sns.color_palette('Blues_d', n_colors=5))
sns.barplot(y_pos, val_counts, palette = p)

plt.xticks(y_pos, ['white', 'black', 'hisp', 'native', 'asian'])
plt.ylabel('Proportion of Deaths')
plt.xlabel('Race')
plt.title('Proportion of Deaths by Race')

plt.show()

#### Model Specification
* Logistic Model 
  * independent variables: 'white', 'black', 'hisp', 'native', 'asian'
  * dependent variable: 'death'

In [ ]:
# bar chart for visualization
GroupedData = df_query.groupby(by='jobsatis').size()
GroupedData.plot.bar(x='lab', y='val', rot=0)
plt.xlabel('jobsatis')
plt.ylabel('observations')
plt.title('Distribution of the Response')

In [ ]:
X = df_query[['female', 'black', 'hisp', 'native', 'asian', 'child', 'youth', 'senior', 'hosp', 'icu', 'medcond']]
# omitted group: male, white, adult
y = df_query['death']

import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

# logit_model=sm.Logit(y,X)
# result=logit_model.fit()
# print(result.summary2())

In [ ]:
X = df_query[['black', 'hisp', 'native', 'asian']]
# omitted group: male, white, adult
y = df_query['icu']

import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
# Close the connection when finished. 
con.close()